# 5-class CNN Classifier for Spoof Images

Imports

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from PIL import Image
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, recall_score, accuracy_score
import numpy as np

File Prep

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
data_dir = '/content/drive/My Drive/preprocessed_data' # Modify this if using your local path
json_path = os.path.join(data_dir, 'test_labels.json')

In [34]:
print(os.listdir(data_dir))

['spoof', 'live', 'test_labels.json']


In [44]:
# Test if working
example_paths = [
    '/content/drive/My Drive/preprocessed_data/spoof/494405.png'
]

for path in example_paths:
    if os.path.exists(path):
        print(f"Exists: {path}")
    else:
        print(f"Missing: {path}")

Exists: /content/drive/My Drive/preprocessed_data/spoof/494405.png


Training the Classifier

In [46]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [47]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [51]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

data_dir = '/content/drive/My Drive/preprocessed_data'
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Manually split dataset
num_train = int(len(dataset) * 0.8)
num_val = len(dataset) - num_train
train_dataset, val_dataset = random_split(dataset, [num_train, num_val])

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False)
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [52]:
# Training loop with recall
for epoch in range(10):  # 10 epochs
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

        print(f'Epoch {epoch+1} {phase} Loss: {epoch_loss:.4f} Recall: {epoch_recall:.4f}')

KeyboardInterrupt: 